### CAFR10

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision
import torch.optim as optim
from datagen import *
from nets import *
from backdoor import Backdoor
from defense import Defense

# move to GPU (if applicable)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/andmholm/Backdoor/backdoor_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Choosing Network and Dataset

In [2]:
# choose network configuration and dataset
backdoor = Backdoor(VggNet, config='16-layer', channels=3, classes=10, dropout=0.5)
backdoor.create_models(optim.SGD, nn.CrossEntropyLoss, lr=0.01)
backdoor.load_data(torchvision.datasets.CIFAR10, ImageEntity)

Files already downloaded and verified
Files already downloaded and verified


### Poison setup

In [3]:
# create patches, target labels, and define transformations/merges 
patches = ImagePatch((9, 9), 3, 'random').get_patches(len(backdoor))
targets = {i : i + 1 if i < 9 else 0 for i in range(10)}
transforms = (None, [RotateTransform()], [LabelTransform(targets)])
merge = ImageMerge(select=True)

### Poisoning

In [4]:
# poison 20% of the data
backdoor.poison(patches, transforms, merge, pct=0.2)

### Training Base and Trojan Net

In [5]:
# train the base DNN & the trojan DNN 
loss = backdoor.train(2, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.99% | Loss: 1.8234 | Samples trained: 12496/50000
Epoch 1/3 | 49.98% | Loss: 1.6271 | Samples trained: 24992/50000
Epoch 1/3 | 74.98% | Loss: 1.4913 | Samples trained: 37488/50000
Epoch 1/3 | 99.97% | Loss: 1.3875 | Samples trained: 49984/50000
Epoch 1 complete | Loss: 1.3878
Epoch 2/3 | 24.99% | Loss: 0.9793 | Samples trained: 12496/50000
Epoch 2/3 | 49.98% | Loss: 0.9366 | Samples trained: 24992/50000
Epoch 2/3 | 74.98% | Loss: 0.8961 | Samples trained: 37488/50000
Epoch 2/3 | 99.97% | Loss: 0.8608 | Samples trained: 49984/50000
Epoch 2 complete | Loss: 0.8610
Epoch 3/3 | 24.99% | Loss: 0.6898 | Samples trained: 12496/50000
Epoch 3/3 | 49.98% | Loss: 0.6737 | Samples trained: 24992/50000
Epoch 3/3 | 74.98% | Loss: 0.6506 | Samples trained: 37488/50000
Epoch 3/3 | 99.97% | Loss: 0.6283 | Samples trained: 49984/50000
Epoch 3 complete | Loss: 0.6285
Training complete | Net Average Loss: 0.9591 | Total epochs: 3
Training started
Epoch 1/3 | 24.99% | Loss: 

### Evaluating Backdoor

In [6]:
# evulate backdoor based on VA & ASR
metrics = backdoor.eval(verbose=True, device=device)

Accuracy on Clean | Base 75.94% | Trojan 75.55% | Difference -0.39%
Base Accuracy on Poison 4.74% | Attack Success Rate (ASR): 64.79%
Average Tensor Distance: 311.00 | Net Tensor Difference 15550167.93


### Creating a Defense

In [7]:
# init the defense
defense = Defense(backdoor)

### Detection setup

In [8]:
# get base & trojan DNN, both clean & poisoned entitysets, sizes ranges to create synthetic patch
base, trojan = backdoor.get_net_modules()
cleantrain, poisontrain, cleantest, poisontest = backdoor.get_datasets()
patch_size_ranges = [(3, 3), (5, 5), (7, 7), (9, 9), (11, 11)]

### Detection Base

In [9]:
# detect backdoor with 30% of the clean samples poisoned and detect based on a 10% drop in VA for base DNN
metrics = defense.detect(base, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 75.38% | Original Accuracy 75.94% | Difference -0.56%
Patch size (5, 5) | Synthetic Poison Accuracy 75.26% | Original Accuracy 75.94% | Difference -0.68%
Patch size (7, 7) | Synthetic Poison Accuracy 72.99% | Original Accuracy 75.94% | Difference -2.95%
Patch size (9, 9) | Synthetic Poison Accuracy 71.56% | Original Accuracy 75.94% | Difference -4.38%
Patch size (11, 11) | Synthetic Poison Accuracy 70.21% | Original Accuracy 75.94% | Difference -5.73%
Average Accuracy 73.08% | Average Difference -2.86% | Lowest Score: 70.21% | Likihood of Backdoor: Low


### Detection Trojan

In [10]:
# detect backdoor with 30% of the clean samples poisoned and detect based on a 10% drop in VA for trojan DNN
metrics = defense.detect(trojan, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 74.94% | Original Accuracy 75.55% | Difference -0.61%
Patch size (5, 5) | Synthetic Poison Accuracy 74.33% | Original Accuracy 75.55% | Difference -1.22%
Patch size (7, 7) | Synthetic Poison Accuracy 71.73% | Original Accuracy 75.55% | Difference -3.82%
Patch size (9, 9) | Synthetic Poison Accuracy 62.13% | Original Accuracy 75.55% | Difference -13.42%
Patch size (11, 11) | Synthetic Poison Accuracy 54.34% | Original Accuracy 75.55% | Difference -21.21%
Average Accuracy 67.49% | Average Difference -8.06% | Lowest Score: 54.34% | Likihood of Backdoor: High


### Blocking Patches setup

In [11]:
# reverse labels (for research VA purposes), create patch to compare variances with in parts of images (same size as patch), define samples to block
patch = ImagePatch((10, 10), 3, 'random')
n = 5000

### Blocking

In [12]:
# block poisoned images with average rgb values the same dimensions as the patch
blockedloader = defense.block(poisontest, patch, n=n)

### Testing after Block Base

In [13]:
# retest after blocking for base DNN
metrics = defense.test(base, blockedloader, verbose=True, device=device)

Testing started
24.84% Testing complete | Loss: 5.6361 | Accuracy: 0.0401
49.68% Testing complete | Loss: 5.5451 | Accuracy: 0.0413
74.52% Testing complete | Loss: 5.4814 | Accuracy: 0.0390
99.36% Testing complete | Loss: 5.4917 | Accuracy: 0.0391
Testing complete | Loss: 5.5041 | Accuracy: 3.92%


### Testing after Block Trojan

In [14]:
# retest after blocking for trojan DNN
metrics = defense.test(trojan, blockedloader, verbose=True, device=device)

Testing started
24.84% Testing complete | Loss: 3.8084 | Accuracy: 0.2556
49.68% Testing complete | Loss: 3.7351 | Accuracy: 0.2552
74.52% Testing complete | Loss: 3.6095 | Accuracy: 0.2647
99.36% Testing complete | Loss: 3.5927 | Accuracy: 0.2692
Testing complete | Loss: 3.6010 | Accuracy: 26.90%


### Defense by retraining

In [15]:
# get a clean dataloader, reset the defense net module, retrain the defense net model (og trojan model)
dataloader = cleantrain.get_dataloader()
defense.reset()
loss = defense.retrain(dataloader, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.95% | Loss: 0.7026 | Samples trained: 12480/50000
Epoch 1/3 | 49.90% | Loss: 0.6933 | Samples trained: 24960/50000
Epoch 1/3 | 74.86% | Loss: 0.6826 | Samples trained: 37440/50000
Epoch 1/3 | 99.81% | Loss: 0.6492 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 0.6489
Epoch 2/3 | 24.95% | Loss: 0.7051 | Samples trained: 12480/50000
Epoch 2/3 | 49.90% | Loss: 0.6947 | Samples trained: 24960/50000
Epoch 2/3 | 74.86% | Loss: 0.6850 | Samples trained: 37440/50000
Epoch 2/3 | 99.81% | Loss: 0.6505 | Samples trained: 49920/50000
Epoch 2 complete | Loss: 0.6501
Epoch 3/3 | 24.95% | Loss: 0.7085 | Samples trained: 12480/50000
Epoch 3/3 | 49.90% | Loss: 0.6948 | Samples trained: 24960/50000
Epoch 3/3 | 74.86% | Loss: 0.6841 | Samples trained: 37440/50000
Epoch 3/3 | 99.81% | Loss: 0.6501 | Samples trained: 49920/50000
Epoch 3 complete | Loss: 0.6498
Training complete | Net Average Loss: 0.6496 | Total epochs: 3


### Evaluation after retraining

In [16]:
# re-evaluate the trojan DNN
cleanloader, poisonloader = cleantest.get_dataloader(), poisontest.get_dataloader()
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 75.94% | Trojan 75.55% | Defense 73.22%
Accuracy on Posion | Base 4.74% | Defense 36.62% | Trojan ASR 64.79%
Difference from Baseline | Trojan -0.39% | Defense -2.72%
Defense Effectiveness | 28.17% decrease in ASR


In [17]:
defense.view_named_modules()

avgpool: AdaptiveAvgPool2d(output_size=(7, 7))
drop: Dropout(p=0.5, inplace=False)
fc1: Linear(in_features=25088, out_features=4096, bias=True)
fc2: Linear(in_features=4096, out_features=4096, bias=True)
fc3: Linear(in_features=4096, out_features=10, bias=True)
layers: Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): MaxPool2d(kernel_size=(2, 2), stride=2, padding=1, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running

### Pruning setup

In [18]:
# determine layers to prune, decide percent of neurons to prune, reset the defense (original trojan DNN)
layers = ['layers.28', 'layers.26', 'layers.24', 'layers.21']
amount = 0.5
defense.reset()

### Pruning based on l-Infinity norm of weights from layers

In [19]:
# prune 50% the weights of last 4 convolutional layers based on the l-Infinity norm
defense.prune(layers, amount)

### Evaluation after prune

In [20]:
# evaluate the model after pruning
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 75.94% | Trojan 75.55% | Defense 10.00%
Accuracy on Posion | Base 4.74% | Defense 10.00% | Trojan ASR 64.79%
Difference from Baseline | Trojan -0.39% | Defense -65.94%
Defense Effectiveness | 54.79% decrease in ASR


### Retraining to recover

In [21]:
# retrain to recover
loss = defense.retrain(dataloader, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.95% | Loss: 2.3223 | Samples trained: 12480/50000
Epoch 1/3 | 49.90% | Loss: 2.3241 | Samples trained: 24960/50000
Epoch 1/3 | 74.86% | Loss: 2.3255 | Samples trained: 37440/50000
Epoch 1/3 | 99.81% | Loss: 2.3251 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 2.3251
Epoch 2/3 | 24.95% | Loss: 2.3229 | Samples trained: 12480/50000
Epoch 2/3 | 49.90% | Loss: 2.3248 | Samples trained: 24960/50000
Epoch 2/3 | 74.86% | Loss: 2.3250 | Samples trained: 37440/50000
Epoch 2/3 | 99.81% | Loss: 2.3249 | Samples trained: 49920/50000
Epoch 2 complete | Loss: 2.3249
Epoch 3/3 | 24.95% | Loss: 2.3220 | Samples trained: 12480/50000
Epoch 3/3 | 49.90% | Loss: 2.3247 | Samples trained: 24960/50000
Epoch 3/3 | 74.86% | Loss: 2.3262 | Samples trained: 37440/50000
Epoch 3/3 | 99.81% | Loss: 2.3262 | Samples trained: 49920/50000
Epoch 3 complete | Loss: 2.3263
Training complete | Net Average Loss: 2.3254 | Total epochs: 3


### Evaluation after retraining

In [22]:
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 75.94% | Trojan 75.55% | Defense 10.00%
Accuracy on Posion | Base 4.74% | Defense 10.00% | Trojan ASR 64.79%
Difference from Baseline | Trojan -0.39% | Defense -65.94%
Defense Effectiveness | 54.79% decrease in ASR
